# Generate average Annotations to compare with Uncertainty masks

In [1]:
# Import libraries
import os
import re
from glob import glob
import numpy as np
from monai.data import CacheDataset, DataLoader
from monai.transforms import AddChanneld, Compose, LoadImaged, ToTensord
from monai.data import write_nifti

In [2]:
# Specify the path to the "eval_in/" directory
annotator_dir = "/home/akamath/Documents/miccaihackathon_shifts/data/shifts_ms_pt1/shifts_ms_pt1/msseg/eval_in/individual_annotators"
save_dir = "/home/akamath/Documents/miccaihackathon_shifts/data/disagreement_in_annotators"
os.makedirs(save_dir, exist_ok=True)

annotator_1_files = sorted(glob(os.path.join(annotator_dir, "annotator1", "*ann1_isovox.nii.gz")),
              key=lambda i: int(re.sub('\D', '', i)))
annotator_2_files = sorted(glob(os.path.join(annotator_dir, "annotator2", "*ann2_isovox.nii.gz")),
              key=lambda i: int(re.sub('\D', '', i)))
annotator_3_files = sorted(glob(os.path.join(annotator_dir, "annotator3", "*ann3_isovox.nii.gz")),
              key=lambda i: int(re.sub('\D', '', i)))
annotator_4_files = sorted(glob(os.path.join(annotator_dir, "annotator4", "*ann4_isovox.nii.gz")),
              key=lambda i: int(re.sub('\D', '', i)))
annotator_5_files = sorted(glob(os.path.join(annotator_dir, "annotator5", "*ann5_isovox.nii.gz")),
              key=lambda i: int(re.sub('\D', '', i)))
annotator_6_files = sorted(glob(os.path.join(annotator_dir, "annotator6", "*ann6_isovox.nii.gz")),
              key=lambda i: int(re.sub('\D', '', i)))
annotator_7_files = sorted(glob(os.path.join(annotator_dir, "annotator7", "*ann7_isovox.nii.gz")),
              key=lambda i: int(re.sub('\D', '', i)))

test_files = [{'image': a1, 'ann2': a2, 'ann3': a3, 'ann4': a4, 'ann5': a5, 'ann6': a6, 'ann7': a7}
              for a1, a2, a3, a4, a5, a6, a7 in zip(annotator_1_files, annotator_2_files,
                                                    annotator_3_files, annotator_4_files,
                                                    annotator_5_files, annotator_6_files,
                                                    annotator_7_files)]

print("Testing cases:", len(test_files))

# Define transforms
val_transforms = Compose(
[
    LoadImaged(keys=["image", "ann2", "ann3", "ann4", "ann5", "ann6", "ann7"]),
    AddChanneld(keys=["image", "ann2", "ann3", "ann4", "ann5", "ann6", "ann7"]),
    ToTensord(keys=["image", "ann2", "ann3", "ann4", "ann5", "ann6", "ann7"]),
]
)
# Define dataset and dataloader to iterate the data
val_ds = CacheDataset(data=test_files, transform=val_transforms,
                      cache_rate=0.1, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=0)

Testing cases: 24


Loading dataset: 100%|██████████| 2/2 [00:01<00:00,  1.63it/s]


In [3]:
for patient_count, batch_data in enumerate(val_loader):
    print("Patient num: ", patient_count + 1)
    ann1_labels, ann2_labels, ann3_labels, \
    ann4_labels, ann5_labels, ann6_labels, \
    ann7_labels = batch_data["image"], \
                  batch_data["ann2"], \
                  batch_data["ann3"], \
                  batch_data["ann4"], \
                  batch_data["ann5"], \
                  batch_data["ann6"], \
                  batch_data["ann7"]

    all_annotators = []
    for idx in range(1, 8):
        if idx == 1:
            ann_data = batch_data["image"].numpy()
        else:
            ann_data = batch_data["ann" + str(idx)].numpy()
        ann_data = np.squeeze(ann_data[0])
        all_annotators.append(ann_data)

    mean_annotator = np.mean(all_annotators, axis=0)
    disagreement_annotator = mean_annotator * (1 - mean_annotator)
    affine=batch_data['image_meta_dict']['original_affine'][0]
    spatial_shape=batch_data['image_meta_dict']['spatial_shape'][0]

    write_nifti(disagreement_annotator,
            os.path.join(save_dir, str(patient_count + 1) + "_disagreement_annotation.nii.gz"),
            affine=affine, target_affine=affine,
            output_spatial_shape=spatial_shape)

Patient num:  1
Patient num:  2
Patient num:  3
Patient num:  4
Patient num:  5
Patient num:  6
Patient num:  7
Patient num:  8
Patient num:  9
Patient num:  10
Patient num:  11
Patient num:  12
Patient num:  13
Patient num:  14
Patient num:  15
Patient num:  16
Patient num:  17
Patient num:  18
Patient num:  19
Patient num:  20
Patient num:  21
Patient num:  22
Patient num:  23
Patient num:  24
